## 꽃 이미지 분류 (VGG19 전이 학습)

이 노트북은 강력한 사전 훈련된 모델인 **VGG19**를 사용하여 꽃 이미지를 분류하는 **전이 학습(Transfer Learning)** 기법을 다룹니다. 전이 학습은 대규모 데이터셋(예: ImageNet)으로 미리 학습된 모델의 지식을 가져와, 더 작은 데이터셋에 맞게 조정하여 사용하는 강력한 방법입니다.

**주요 과정:**
1.  **데이터셋 재구성**: 원본 데이터셋을 `train`과 `test` 폴더로 분리하여 모델 학습에 적합한 구조로 만듭니다.
2.  **데이터 로딩**: Keras의 `image_dataset_from_directory` 유틸리티를 사용하여 디스크에서 직접 이미지를 효율적으로 로드하고, 훈련/검증 세트를 나눕니다.
3.  **VGG19 모델 로드**: ImageNet으로 사전 학습된 VGG19 모델의 합성곱 기반(convolutional base)을 불러옵니다.
4.  **모델 동결**: 불러온 VGG19의 가중치가 훈련 중에 업데이트되지 않도록 동결(freeze)합니다.
5.  **데이터 증강**: 훈련 데이터에 무작위 변환을 적용하여 과대적합을 줄이고 모델의 일반화 성능을 높입니다.
6.  **새로운 모델 구축**: 동결된 VGG19 기반 위에 새로운 분류기(classifier)를 추가하여 최종 모델을 완성합니다.
7.  **모델 훈련 및 평가**: 구축된 모델을 훈련하고, 테스트 데이터셋으로 성능을 평가합니다.

### 1. 라이브러리 임포트 및 경로 설정

In [13]:
import os
import shutil
import pathlib
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers, models
from keras.utils import image_dataset_from_directory
import matplotlib.pyplot as plt
import pickle

# 경로 설정
original_dir = pathlib.Path("../data/flowers")
new_base_dir = pathlib.Path("../data/new_flowers")

### 2. 데이터셋 재구성
Keras의 데이터 로딩 유틸리티를 효율적으로 사용하기 위해, 원본 데이터셋을 `train`과 `test` 하위 디렉토리로 복사하여 재구성합니다. 이 작업은 한 번만 수행하면 됩니다.

In [14]:
def make_subset(subset_name, start_index, end_index):
    """주어진 인덱스 범위에 따라 데이터의 서브셋을 만듭니다."""
    for category in ("daisy", "dandelion", "tulip", "rose", "sunflower"):
        dir_path = new_base_dir / subset_name / category
        os.makedirs(dir_path, exist_ok=True)
        
        source_dir = original_dir / category
        data_list = [f for f in os.listdir(source_dir) if os.path.isfile(source_dir / f)]
        
        if end_index is not None:
            fnames = data_list[start_index:end_index]
        else:
            fnames = data_list[start_index:]
            
        for fname in fnames:
            shutil.copyfile(src=source_dir / fname, dst=dir_path / fname)

# new_base_dir이 이미 존재하면 재구성 작업을 건너뜁니다.
if not os.path.exists(new_base_dir):
    print(f"'{new_base_dir}'를 생성하고 데이터를 재구성합니다...")
    # 각 클래스별로 700개 이미지를 훈련용으로, 나머지를 테스트용으로 분리
    make_subset("train", 0, 700)
    make_subset("test", 700, None)
    print("데이터 재구성 완료.")
else:
    print(f"'{new_base_dir}'가 이미 존재합니다. 데이터 재구성을 건너뜁니다.")

'..\data\new_flowers'가 이미 존재합니다. 데이터 재구성을 건너뜁니다.


### 3. TensorFlow 데이터셋 생성
`image_dataset_from_directory`를 사용하여 디스크에서 직접 이미지를 로드하는 `tf.data.Dataset` 객체를 생성합니다. 이 방식은 메모리를 효율적으로 사용하게 해줍니다.

- 훈련 데이터(`train_ds`)는 다시 8:2 비율로 나뉘어 실제 훈련용과 검증용(`validation_ds`)으로 사용됩니다.

In [15]:
# 훈련 데이터셋 (80%)
train_ds = image_dataset_from_directory(
    new_base_dir / "train",
    seed=1234,
    subset='training',
    validation_split=0.2,
    image_size=(180, 180),
    batch_size=16
)

# 검증 데이터셋 (20%)
validation_ds = image_dataset_from_directory(
    new_base_dir / "train",
    seed=1234,
    subset='validation',
    validation_split=0.2,
    image_size=(180, 180),
    batch_size=16
)

# 테스트 데이터셋
test_ds = image_dataset_from_directory(
    new_base_dir / "test",
    image_size=(180, 180),
    batch_size=16
)

print("\n클래스 이름:", train_ds.class_names)

Found 0 files belonging to 1 classes.
Using 0 files for training.


ValueError: No images found in directory ..\data\new_flowers\train. Allowed formats: ('.bmp', '.gif', '.jpeg', '.jpg', '.png')

### 4. VGG19 모델 로드 및 동결
ImageNet으로 사전 학습된 VGG19 모델의 합성곱 부분만 불러옵니다 (`include_top=False`). 그 후, 이 사전 학습된 가중치들이 훈련 중에 변경되지 않도록 동결합니다.

In [ ]:
conv_base = keras.applications.vgg19.VGG19(
    weights="imagenet",
    include_top=False,
    input_shape=(180, 180, 3)
)

print("동결 전 훈련 가능 가중치 수:", len(conv_base.trainable_weights))
conv_base.trainable = False
print("동결 후 훈련 가능 가중치 수:", len(conv_base.trainable_weights))

### 5. 데이터 증강 및 최종 모델 구축
과대적합을 방지하기 위해 데이터 증강 레이어를 정의합니다. 그 후, Keras의 함수형 API를 사용하여 VGG19 베이스와 새로운 분류기를 연결하여 최종 모델을 만듭니다.

In [ ]:
# 데이터 증강 레이어
data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.4)
])

# 모델 구축
inputs = keras.Input(shape=(180, 180, 3))
x = data_augmentation(inputs) # 1. 데이터 증강
x = keras.applications.vgg19.preprocess_input(x) # 2. VGG19에 맞는 전처리
x = conv_base(x) # 3. VGG19 기반으로 특징 추출
x = layers.Flatten()(x) # 4. 1차원으로 펼치기
x = layers.Dense(256)(x) # 5. 새로운 분류기
x = layers.Dense(128)(x)
x = layers.Dense(64)(x)
outputs = layers.Dense(5, activation='softmax')(x) # 6. 최종 출력층

model = keras.Model(inputs, outputs)

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.summary()

### 6. 모델 훈련
`ModelCheckpoint` 콜백을 사용하여 검증 손실이 가장 낮은 최적의 모델을 저장하면서 훈련을 진행합니다.

In [ ]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="flowers_vgg19.keras",
        save_best_only=True,
        monitor='val_loss'
    )
]

history = model.fit(train_ds,
                    epochs=5, # 데모를 위해 에포크 수를 줄임 (원래는 더 많이 필요)
                    validation_data=validation_ds,
                    callbacks=callbacks)

# 훈련 기록 저장
with open("flowers_vgg19_history.pkl", "wb") as file:
    pickle.dump(history.history, file)

### 7. 모델 평가 및 결과 시각화
훈련된 모델의 최종 성능을 평가하고, 훈련 과정 동안의 손실과 정확도 변화를 그래프로 시각화합니다.

In [ ]:
# 훈련 과정 시각화
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# 저장된 최적 모델 로드 및 평가
print("\n--- 최종 모델 평가 (테스트 데이터셋) ---")
best_model = keras.models.load_model("flowers_vgg19.keras")
test_loss, test_acc = best_model.evaluate(test_ds)
print(f"테스트 손실: {test_loss:.4f}")
print(f"테스트 정확도: {test_acc:.4f}")